##Obtain all subforums and forum urls from Careers and Jobs page

In [3]:
import requests
import warnings
from bs4 import BeautifulSoup
warnings.filterwarnings('ignore')

base_url = "https://www.thestudentroom.co.uk/"
add_url = "&page={}" #For setting page
careers_and_jobs_url = "https://www.thestudentroom.co.uk/forumdisplay.php?f=201"
all_career_and_jobs_urls = []

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
}
response = requests.get(careers_and_jobs_url, headers=headers)
if response.status_code == 200:
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    forum_info_cells = soup.find_all('td', class_='info')

    career_and_jobs_url = []

    for cell in forum_info_cells:
        main_forum_tag = cell.find('a', href=True)
        if main_forum_tag:
            print(base_url + main_forum_tag['href'])
            main_forum_link = base_url + main_forum_tag['href'] + add_url
            main_forum_title = main_forum_tag.text.strip()
            career_and_jobs_url.append(main_forum_link)

        subforum_section = cell.find('div', class_='subforums')
        if subforum_section:
            subforum_links = subforum_section.find_all('a', href=True)
            for subforum_tag in subforum_links:
                print(base_url + subforum_tag['href'])
                subforum_link = base_url + subforum_tag['href'] + add_url
                subforum_title = subforum_tag.text.strip()
                career_and_jobs_url.append(subforum_link)

    for url in career_and_jobs_url:
        all_career_and_jobs_urls.append(url)
else:
    print(f"Failed to retrieve data. Status Code: {response.status_code}")


https://www.thestudentroom.co.uk/forumdisplay.php?f=1610
https://www.thestudentroom.co.uk/forumdisplay.php?f=1611
https://www.thestudentroom.co.uk/forumdisplay.php?f=1313
https://www.thestudentroom.co.uk/forumdisplay.php?f=339
https://www.thestudentroom.co.uk/forumdisplay.php?f=8
https://www.thestudentroom.co.uk/forumdisplay.php?f=1612
https://www.thestudentroom.co.uk/forumdisplay.php?f=1613
https://www.thestudentroom.co.uk/forumdisplay.php?f=94
https://www.thestudentroom.co.uk/forumdisplay.php?f=97
https://www.thestudentroom.co.uk/forumdisplay.php?f=262
https://www.thestudentroom.co.uk/forumdisplay.php?f=263
https://www.thestudentroom.co.uk/forumdisplay.php?f=513
https://www.thestudentroom.co.uk/forumdisplay.php?f=96
https://www.thestudentroom.co.uk/forumdisplay.php?f=673
https://www.thestudentroom.co.uk/forumdisplay.php?f=512
https://www.thestudentroom.co.uk/forumdisplay.php?f=672
https://www.thestudentroom.co.uk/forumdisplay.php?f=148
https://www.thestudentroom.co.uk/forumdisplay.ph

##Create list of links, post title, replies and post timestamp for all category pages

We will scrape a 25k posts or more per category for training our model, however, due to time constraints, this notebook will scrape only 5k posts per category.

For training: With every page containing approximately 80 posts, we can scrape 25,000/80 = 312 pages per category. This limits the time taken for scraping and prevent memory constraints. We will be retaining newer pages, as the majority of posts would be considered "old".

Category: A Level

In [ ]:
from typing_extensions import final
import pandas as pd

a_level_url = "https://www.thestudentroom.co.uk/forumdisplay.php?f=80&page={}"
a_level_df = pd.DataFrame(columns=["Title", "Replies", "Link", "Date", "Category", "Post Content Start"])

rows = []

import time
start = time.time()
final_page = 0

for page in range(312):
    cur_page_url = a_level_url.format(page)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
    }
    response = requests.get(cur_page_url, headers=headers)

    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        threads = soup.find_all('tr', class_='thread')

        for thread in threads:
            if 'sticky' in thread.get('class', []):
                continue

            title_tag = thread.find('a', id=lambda x: x and x.startswith('thread_title_'))
            title = title_tag.text if title_tag else 'No title found'
            link = base_url + title_tag['href'] if title_tag else 'No link found'

            replies_tag = thread.find('span', class_='replies')
            replies = replies_tag.text.strip().replace('Replies:', '').strip() if replies_tag else '0'

            date_tag = thread.find('span', class_='date_text') or thread.find('span', class_='mobile_date_text')
            date = date_tag.text if date_tag else 'No date found'

            # Extracting the post content start (beginning of the post)
            post_content_tag = thread.find('td', class_='title')
            post_content_start = post_content_tag['title'] if post_content_tag and 'title' in post_content_tag.attrs else 'No content start found'

            rows.append({
                "Title": title,
                "Replies": replies,
                "Link": link,
                "Date": date,
                "Category": "A Level",
                "Post Content Start": post_content_start
            })

    elif response.status_code == 404:
        print(f"All pages scraped. Last page: {page+1}"); break
    else:
        print(f"Failed to retrieve data. Status Code: {response.status_code}")


stop = time.time()
duration = stop-start
print(f"Duration of scrape: {duration}s")
a_level_df = pd.concat([a_level_df, pd.DataFrame(rows)], ignore_index=True)
a_level_df.to_csv("a_level.csv", index=False)
a_level_df

Duration of scrape: 894.3866891860962s


,Title,Replies,Link,Date,Category,Post Content Start
0,changing a levels,1,https://www.thestudentroom.co.uk/showthread.ph...,1 minute ago,A Level,hi so i’m currently doing psychology film and...
1,"Programs, Courses, Extracurriculars, and Compe...",2,https://www.thestudentroom.co.uk/showthread.ph...,2 minutes ago,A Level,"Hi everyone, \n \nI’m currently in Year 11, an..."
2,English literature or medicine,6,https://www.thestudentroom.co.uk/showthread.ph...,3 minutes ago,A Level,Hey yall do im in a really big dilema! \nIm i...
3,Alevels maths,11,https://www.thestudentroom.co.uk/showthread.ph...,8 minutes ago,A Level,"I am a year 12 student , I just sat my year 12..."
4,Home student wanting to do CIE A-levels in nov,1,https://www.thestudentroom.co.uk/showthread.ph...,10 minutes ago,A Level,"Hi everyone, \n \nI’m a UK home student studyi..."
...,...,...,...,...,...,...
24955,Do I need to do Further Maths at GCSE for A-Le...,6,https://www.thestudentroom.co.uk/showthread.ph...,02-12-2021,A Level,Hey guys!! \nI want to do A-Level maths and my...
24956,N/A,8,https://www.thestudentroom.co.uk/showthread.ph...,02-12-2021,A Level,What to do if your teacher can't mark properly?
24957,Non-STEM A level choices but wanting to study CS?,3,https://www.thestudentroom.co.uk/showthread.ph...,02-12-2021,A Level,I’m really passionate about and enjoy essay-ba...
24958,papers,1,https://www.thestudentroom.co.uk/showthread.ph...,02-12-2021,A Level,how do l get physics papers


Category: Career and Jobs

In [5]:
import pandas as pd

job_df = pd.DataFrame(columns=["Title", "Replies", "Link", "Date", "Category", "Post Content Start"])

job_rows = []
flag = 0 # For detecting private threads - If 3 or more pages have no threads, we skip directly to the next link

for link in all_career_and_jobs_urls:
    print("Cur Link: ", link)
    for page in range(312):
        if flag >= 3:
          break
        cur_page_url = link.format(page)
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
        }
        response = requests.get(cur_page_url, headers=headers)

        if response.status_code == 200:
            html_content = response.text
            soup = BeautifulSoup(html_content, 'html.parser')

            threads = soup.find_all('tr', class_='thread')
            if not threads:
              flag += 1

            for thread in threads:
                if 'sticky' in thread.get('class', []):
                    continue

                title_tag = thread.find('a', id=lambda x: x and x.startswith('thread_title_'))
                title = title_tag.text if title_tag else 'No title found'
                thread_link = base_url + title_tag['href'] if title_tag else 'No link found'

                replies_tag = thread.find('span', class_='replies')
                replies = replies_tag.text.strip().replace('Replies:', '').strip() if replies_tag else '0'

                date_tag = thread.find('span', class_='date_text') or thread.find('span', class_='mobile_date_text')
                date = date_tag.text if date_tag else 'No date found'

                # Extracting the post content start (beginning of the post)
                post_content_tag = thread.find('td', class_='title')
                post_content_start = post_content_tag['title'] if post_content_tag and 'title' in post_content_tag.attrs else 'No content start found'

                job_rows.append({
                    "Title": title,
                    "Replies": replies,
                    "Link": thread_link,
                    "Date": date,
                    "Category": "Job Experience",
                    "Post Content Start": post_content_start
                })

        elif response.status_code == 404:
            print(f"All pages scraped. Last page: {page}"); break
        else:
            print(f"Failed to retrieve data. Status Code: {response.status_code}")

job_df = pd.concat([job_df, pd.DataFrame(job_rows)], ignore_index=True)
job_df.to_csv("job.csv", index=False)
job_df

Cur Link:  https://www.thestudentroom.co.uk/forumdisplay.php?f=1610&page={}
Failed to retrieve data. Status Code: 500
Failed to retrieve data. Status Code: 500
Cur Link:  https://www.thestudentroom.co.uk/forumdisplay.php?f=1611&page={}
Cur Link:  https://www.thestudentroom.co.uk/forumdisplay.php?f=1313&page={}
Cur Link:  https://www.thestudentroom.co.uk/forumdisplay.php?f=339&page={}
Cur Link:  https://www.thestudentroom.co.uk/forumdisplay.php?f=8&page={}
Cur Link:  https://www.thestudentroom.co.uk/forumdisplay.php?f=1612&page={}
Cur Link:  https://www.thestudentroom.co.uk/forumdisplay.php?f=1613&page={}
Cur Link:  https://www.thestudentroom.co.uk/forumdisplay.php?f=94&page={}
Cur Link:  https://www.thestudentroom.co.uk/forumdisplay.php?f=97&page={}
Cur Link:  https://www.thestudentroom.co.uk/forumdisplay.php?f=262&page={}
Cur Link:  https://www.thestudentroom.co.uk/forumdisplay.php?f=263&page={}
Cur Link:  https://www.thestudentroom.co.uk/forumdisplay.php?f=513&page={}
Cur Link:  http

,Title,Replies,Link,Date,Category,Post Content Start
0,Coworker got another raise while I am still st...,2,https://www.thestudentroom.co.uk/showthread.ph...,3 days ago,Job Experience,I know I will be hammered for this post but I...
1,"Digital marketing, where to start?",12,https://www.thestudentroom.co.uk/showthread.ph...,1 week ago,Job Experience,I’m a 26 year old with ADHD that went into...
2,Whats the state of UK job market in 2024/2025?,7,https://www.thestudentroom.co.uk/showthread.ph...,1 week ago,Job Experience,I have sent out 80+ application and i only got...
3,Have I been ghosted?,4,https://www.thestudentroom.co.uk/showthread.ph...,3 weeks ago,Job Experience,I had an interview at the company. I thought t...
4,Is it bad to leave a 2 year contract after a y...,3,https://www.thestudentroom.co.uk/showthread.ph...,3 weeks ago,Job Experience,I graduated with a computer Science degree in...
...,...,...,...,...,...,...
49919,Anyone else get a call for a job at Chessingto...,0,https://www.thestudentroom.co.uk/showthread.ph...,21-10-2015,Job Experience,Hi everyone. Just wondering if anyone else got...
49920,"Job Interview at Omniplex cinema, help?",0,https://www.thestudentroom.co.uk/showthread.ph...,18-10-2015,Job Experience,hey ive got a job interview at an omniplex cin...
49921,Numerical Reasoning Test - Weak Point,2,https://www.thestudentroom.co.uk/showthread.ph...,16-10-2015,Job Experience,"Guys, \n \nI have graduated a couple months ag..."
49922,Pets At Home Group Assessment?,0,https://www.thestudentroom.co.uk/showthread.ph...,17-07-2014,Job Experience,I have a group assessment for Pets at Home soo...


In [6]:
job_df

,Title,Replies,Link,Date,Category,Post Content Start
0,Coworker got another raise while I am still st...,2,https://www.thestudentroom.co.uk/showthread.ph...,3 days ago,Job Experience,I know I will be hammered for this post but I...
1,"Digital marketing, where to start?",12,https://www.thestudentroom.co.uk/showthread.ph...,1 week ago,Job Experience,I’m a 26 year old with ADHD that went into...
2,Whats the state of UK job market in 2024/2025?,7,https://www.thestudentroom.co.uk/showthread.ph...,1 week ago,Job Experience,I have sent out 80+ application and i only got...
3,Have I been ghosted?,4,https://www.thestudentroom.co.uk/showthread.ph...,3 weeks ago,Job Experience,I had an interview at the company. I thought t...
4,Is it bad to leave a 2 year contract after a y...,3,https://www.thestudentroom.co.uk/showthread.ph...,3 weeks ago,Job Experience,I graduated with a computer Science degree in...
...,...,...,...,...,...,...
49919,Anyone else get a call for a job at Chessingto...,0,https://www.thestudentroom.co.uk/showthread.ph...,21-10-2015,Job Experience,Hi everyone. Just wondering if anyone else got...
49920,"Job Interview at Omniplex cinema, help?",0,https://www.thestudentroom.co.uk/showthread.ph...,18-10-2015,Job Experience,hey ive got a job interview at an omniplex cin...
49921,Numerical Reasoning Test - Weak Point,2,https://www.thestudentroom.co.uk/showthread.ph...,16-10-2015,Job Experience,"Guys, \n \nI have graduated a couple months ag..."
49922,Pets At Home Group Assessment?,0,https://www.thestudentroom.co.uk/showthread.ph...,17-07-2014,Job Experience,I have a group assessment for Pets at Home soo...


Category: Studying, Revision and Exam Support

In [ ]:
study_support_url = "https://www.thestudentroom.co.uk/forumdisplay.php?f=635&page={}"
study_support_df = pd.DataFrame(columns=["Title", "Replies", "Link", "Date", "Category", "Post Content Start"])

rows = []

for page in range(312):
    cur_page_url = study_support_url.format(page)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
    }
    response = requests.get(cur_page_url, headers=headers)

    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        threads = soup.find_all('tr', class_='thread')

        for thread in threads:
            if 'sticky' in thread.get('class', []):
                continue

            title_tag = thread.find('a', id=lambda x: x and x.startswith('thread_title_'))
            title = title_tag.text if title_tag else 'No title found'
            link = base_url + title_tag['href'] if title_tag else 'No link found'

            replies_tag = thread.find('span', class_='replies')
            replies = replies_tag.text.strip().replace('Replies:', '').strip() if replies_tag else '0'

            date_tag = thread.find('span', class_='date_text') or thread.find('span', class_='mobile_date_text')
            date = date_tag.text if date_tag else 'No date found'

            # Extracting the post content start (beginning of the post)
            post_content_tag = thread.find('td', class_='title')
            post_content_start = post_content_tag['title'] if post_content_tag and 'title' in post_content_tag.attrs else 'No content start found'

            rows.append({
                "Title": title,
                "Replies": replies,
                "Link": link,
                "Date": date,
                "Category": "Study Support",
                "Post Content Start": post_content_start
            })

    elif response.status_code == 404:
        print(f"All pages scraped. Last page: {page}"); break
    else:
        print(f"Failed to retrieve data. Status Code: {response.status_code}")

study_support_df = pd.concat([study_support_df, pd.DataFrame(rows)], ignore_index=True)
study_support_df.to_csv("study_support.csv", index=False)
study_support_df

,Title,Replies,Link,Date,Category,Post Content Start
0,Study focus,0,https://www.thestudentroom.co.uk/showthread.ph...,10 hours ago,Study Support,Help !
1,Revision of a level business,0,https://www.thestudentroom.co.uk/showthread.ph...,22 hours ago,Study Support,"Hey there, just wanted to ask about what revis..."
2,unit 5 chemistry (btec applied science),8,https://www.thestudentroom.co.uk/showthread.ph...,1 day ago,Study Support,has anyone found a good way to revise chemistr...
3,Does anyone know what poem was on English Lite...,3,https://www.thestudentroom.co.uk/showthread.ph...,1 day ago,Study Support,I want to prepare for my mock.
4,GCSE Revision Timetable,2,https://www.thestudentroom.co.uk/showthread.ph...,1 day ago,Study Support,I struggle with revision timetables quite a bi...
...,...,...,...,...,...,...
20275,Lilians's revision tips [TSR video panel],0,https://www.thestudentroom.co.uk/showthread.ph...,21-04-2011,Study Support,4KRzYtES-24 \n \nLilian gives her top revision...
20276,Megan's revision tips [TSR video panel],0,https://www.thestudentroom.co.uk/showthread.ph...,21-04-2011,Study Support,KLUo6soBUfU \n \nMegan gives her top revision ...
20277,Nick's revision tips [TSR video panel],0,https://www.thestudentroom.co.uk/showthread.ph...,21-04-2011,Study Support,fjQaQMga9IY \n \nNick gives his top revision t...
20278,Rory's revision tips [TSR video panel],0,https://www.thestudentroom.co.uk/showthread.ph...,21-04-2011,Study Support,3BFeHtuJz50 \n \nRory gives his top revision t...


Category: GCSE

In [8]:
gcse_url = "https://www.thestudentroom.co.uk/forumdisplay.php?f=85&page={}"
gcse_df = pd.DataFrame(columns=["Title", "Replies", "Link", "Date", "Category", "Post Content Start"])

rows = []

for page in range(312):
    cur_page_url = gcse_url.format(page)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
    }
    response = requests.get(cur_page_url, headers=headers)

    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        threads = soup.find_all('tr', class_='thread')

        for thread in threads:
            if 'sticky' in thread.get('class', []):
                continue

            title_tag = thread.find('a', id=lambda x: x and x.startswith('thread_title_'))
            title = title_tag.text if title_tag else 'No title found'
            link = base_url + title_tag['href'] if title_tag else 'No link found'

            replies_tag = thread.find('span', class_='replies')
            replies = replies_tag.text.strip().replace('Replies:', '').strip() if replies_tag else '0'

            date_tag = thread.find('span', class_='date_text') or thread.find('span', class_='mobile_date_text')
            date = date_tag.text if date_tag else 'No date found'

            # Extracting the post content start (beginning of the post)
            post_content_tag = thread.find('td', class_='title')
            post_content_start = post_content_tag['title'] if post_content_tag and 'title' in post_content_tag.attrs else 'No content start found'

            # Adding the data to rows
            rows.append({
                "Title": title,
                "Replies": replies,
                "Link": link,
                "Date": date,
                "Category": "GCSE",
                "Post Content Start": post_content_start
            })

    elif response.status_code == 404:
        print(f"All pages scraped. Last page: {page}")
        break
    else:
        print(f"Failed to retrieve data. Status Code: {response.status_code}")

gcse_df = pd.concat([gcse_df, pd.DataFrame(rows)], ignore_index=True)
gcse_df.to_csv("gcse.csv", index=False)

gcse_df

,Title,Replies,Link,Date,Category,Post Content Start
0,Economics Igcse,0,https://www.thestudentroom.co.uk/showthread.ph...,31 minutes ago,GCSE,Hi I'm currently taking the edexcel economics...
1,GCSE 2004 born 1987/88 Exam board Essex,1,https://www.thestudentroom.co.uk/showthread.ph...,2 hours ago,GCSE,Does anyone know which exam board they sat the...
2,Need help with inconsistent motivation,1,https://www.thestudentroom.co.uk/showthread.ph...,3 hours ago,GCSE,Hiya. I'm in Year 11 and I did 3 GCSE exams in...
3,"I got 99999999888 at GCSE, ask me anything ‼️",29,https://www.thestudentroom.co.uk/showthread.ph...,3 hours ago,GCSE,I got: \n \nMaths - 9 \nPhysics-9 \nChemistry...
4,How to study English,11,https://www.thestudentroom.co.uk/showthread.ph...,5 hours ago,GCSE,I am getting 3s but I need to get at least 5s
...,...,...,...,...,...,...
24955,A few questions about ICT Igcse,1,https://www.thestudentroom.co.uk/showthread.ph...,19-09-2020,GCSE,I'll be sitting my ict exam next month and i h...
24956,need help with ict igcse!!! 17 days left!,1,https://www.thestudentroom.co.uk/showthread.ph...,19-09-2020,GCSE,"i have my ict practical papers(P2,P3) in 17..."
24957,GCSE Revision Website,8,https://www.thestudentroom.co.uk/showthread.ph...,19-09-2020,GCSE,I'm thinking to create a website that helps...
24958,I would like to get a 9 in History GCSE- HELLP...,3,https://www.thestudentroom.co.uk/showthread.ph...,18-09-2020,GCSE,"Hello Everyone, I'm on a grade 3 for GCSE Hist..."


##Combine posts

In [9]:
a_level_df = pd.read_csv("drive/MyDrive/School/Y3S2/AAP/a_level.csv")
job_df = pd.read_csv("drive/MyDrive/School/Y3S2/AAP/job.csv")
study_support_df = pd.read_csv("drive/MyDrive/School/Y3S2/AAP/study_support.csv")

In [10]:
combined_df = pd.concat([gcse_df, a_level_df, job_df, study_support_df], ignore_index=True)
combined_df.to_csv("combined_posts.csv", index=False) #Save file to avoid losing data
combined_df

,Title,Replies,Link,Date,Category,Post Content Start
0,Economics Igcse,0,https://www.thestudentroom.co.uk/showthread.ph...,31 minutes ago,GCSE,Hi I'm currently taking the edexcel economics...
1,GCSE 2004 born 1987/88 Exam board Essex,1,https://www.thestudentroom.co.uk/showthread.ph...,2 hours ago,GCSE,Does anyone know which exam board they sat the...
2,Need help with inconsistent motivation,1,https://www.thestudentroom.co.uk/showthread.ph...,3 hours ago,GCSE,Hiya. I'm in Year 11 and I did 3 GCSE exams in...
3,"I got 99999999888 at GCSE, ask me anything ‼️",29,https://www.thestudentroom.co.uk/showthread.ph...,3 hours ago,GCSE,I got: \n \nMaths - 9 \nPhysics-9 \nChemistry...
4,How to study English,11,https://www.thestudentroom.co.uk/showthread.ph...,5 hours ago,GCSE,I am getting 3s but I need to get at least 5s
...,...,...,...,...,...,...
120119,Lilians's revision tips [TSR video panel],0,https://www.thestudentroom.co.uk/showthread.ph...,21-04-2011,Study Support,4KRzYtES-24 \n \nLilian gives her top revision...
120120,Megan's revision tips [TSR video panel],0,https://www.thestudentroom.co.uk/showthread.ph...,21-04-2011,Study Support,KLUo6soBUfU \n \nMegan gives her top revision ...
120121,Nick's revision tips [TSR video panel],0,https://www.thestudentroom.co.uk/showthread.ph...,21-04-2011,Study Support,fjQaQMga9IY \n \nNick gives his top revision t...
120122,Rory's revision tips [TSR video panel],0,https://www.thestudentroom.co.uk/showthread.ph...,21-04-2011,Study Support,3BFeHtuJz50 \n \nRory gives his top revision t...


In [11]:
gcse_df["Link"][0]

'https://www.thestudentroom.co.uk/showthread.php?t=7560770'

##Enter links to obtain post content and likes, then append to dataframe.

In [ ]:
# import re

# start_index = 0

# # Loop starts from the specific index
# for index, row in combined_df.iloc[start_index:].iterrows():
#     content_start = row["Post Content Start"]
#     url = row["Link"]
#     response = requests.get(url)

#     if response.status_code == 200:
#         response_text = response.text

#         # Regex to match 'rep_count' and 'content' in the response
#         match = re.search(r'"rep_count":(\d+),"content":"(.*?)","has_attachments":', response_text)

#         if match:
#             rep_count = match.group(1)
#             post_content = match.group(2)

#             combined_df.at[index, 'Rep Count'] = rep_count
#             combined_df.at[index, 'Post Content'] = post_content

#         else:
#             print(f"Failed to extract content or rep_count for URL: {url}")

#     else:
#         print(f"Failed to retrieve data for URL: {url}. Status Code: {response.status_code}")

# combined_df.to_csv("combined_posts_final.csv", index=False)


Asynchronous Processing is much faster

In [12]:
import aiohttp
import asyncio
import re
from aiohttp import ClientTimeout

async def fetch_post_data(session, index, row):
    content_start = row["Post Content Start"]
    url = row["Link"]

    timeout = ClientTimeout(total=7200, connect=7200)

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive"
    }

    try:
        async with session.get(url, headers = headers, timeout=timeout) as response:
            if response.status == 200:
                response_text = await response.text()

                match = re.search(r'"rep_count":(\d+),"content":"(.*?)","has_attachments":', response_text)

                if match:
                    rep_count = match.group(1)
                    post_content = match.group(2)
                    return index, rep_count, post_content
                else:
                    print(f"Failed to extract content or rep_count for URL: {url}")
                    return index, None, None
            else:
                print(f"Failed to retrieve data for URL: {url}. Status Code: {response.status}")
                return index, None, None
    except aiohttp.ClientError as e:
        print(f"Error occurred with URL {url}: {e}")
        return index, None, None
    except asyncio.TimeoutError:
        print(f"Request timed out for URL: {url}")
        return index, None, None

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for index, row in combined_df.iterrows():
            task = fetch_post_data(session, index, row)
            tasks.append(task)

        results = await asyncio.gather(*tasks)

        # After all tasks complete, update the DataFrame
        for index, rep_count, post_content in results:
            if rep_count is not None and post_content is not None:
                combined_df.at[index, 'Rep Count'] = rep_count
                combined_df.at[index, 'Post Content'] = post_content

await main()


Failed to retrieve data for URL: https://www.thestudentroom.co.uk/showthread.php?t=7531263. Status Code: 504
Failed to retrieve data for URL: https://www.thestudentroom.co.uk/showthread.php?t=7537884. Status Code: 504
Failed to retrieve data for URL: https://www.thestudentroom.co.uk/showthread.php?t=7537187. Status Code: 504
Failed to retrieve data for URL: https://www.thestudentroom.co.uk/showthread.php?t=7537544. Status Code: 504
Failed to retrieve data for URL: https://www.thestudentroom.co.uk/showthread.php?t=7444913. Status Code: 504
Failed to retrieve data for URL: https://www.thestudentroom.co.uk/showthread.php?t=7534122. Status Code: 504
Failed to retrieve data for URL: https://www.thestudentroom.co.uk/showthread.php?t=7536252. Status Code: 504
Failed to retrieve data for URL: https://www.thestudentroom.co.uk/showthread.php?t=7536217. Status Code: 504
Failed to retrieve data for URL: https://www.thestudentroom.co.uk/showthread.php?t=7536752. Status Code: 504
Failed to retrieve 

In [13]:
combined_df

,Title,Replies,Link,Date,Category,Post Content Start,Rep Count,Post Content
0,Economics Igcse,0,https://www.thestudentroom.co.uk/showthread.ph...,31 minutes ago,GCSE,Hi I'm currently taking the edexcel economics...,0,Hi I'm currently taking the edexcel economics ...
1,GCSE 2004 born 1987/88 Exam board Essex,1,https://www.thestudentroom.co.uk/showthread.ph...,2 hours ago,GCSE,Does anyone know which exam board they sat the...,0,Does anyone know which exam board they sat the...
2,Need help with inconsistent motivation,1,https://www.thestudentroom.co.uk/showthread.ph...,3 hours ago,GCSE,Hiya. I'm in Year 11 and I did 3 GCSE exams in...,0,Hiya. I'm in Year 11 and I did 3 GCSE exams in...
3,"I got 99999999888 at GCSE, ask me anything ‼️",29,https://www.thestudentroom.co.uk/showthread.ph...,3 hours ago,GCSE,I got: \n \nMaths - 9 \nPhysics-9 \nChemistry...,2,I got:\n\nMaths - 9\nPhysics-9 \nChemistry- 9 ...
4,How to study English,11,https://www.thestudentroom.co.uk/showthread.ph...,5 hours ago,GCSE,I am getting 3s but I need to get at least 5s,1,I am getting 3s but I need to get at least 5s
...,...,...,...,...,...,...,...,...
120119,Lilians's revision tips [TSR video panel],0,https://www.thestudentroom.co.uk/showthread.ph...,21-04-2011,Study Support,4KRzYtES-24 \n \nLilian gives her top revision...,1,[CENTER][youtube]4KRzYtES-24[/youtube]\n\nLili...
120120,Megan's revision tips [TSR video panel],0,https://www.thestudentroom.co.uk/showthread.ph...,21-04-2011,Study Support,KLUo6soBUfU \n \nMegan gives her top revision ...,0,[CENTER][youtube]KLUo6soBUfU[/youtube]\n\nMega...
120121,Nick's revision tips [TSR video panel],0,https://www.thestudentroom.co.uk/showthread.ph...,21-04-2011,Study Support,fjQaQMga9IY \n \nNick gives his top revision t...,1,[CENTER][youtube]fjQaQMga9IY[/youtube]\n\nNick...
120122,Rory's revision tips [TSR video panel],0,https://www.thestudentroom.co.uk/showthread.ph...,21-04-2011,Study Support,3BFeHtuJz50 \n \nRory gives his top revision t...,9,[CENTER][youtube]3BFeHtuJz50[/youtube]\n\nRory...


In [14]:
combined_df.to_csv("combined_posts_final.csv", index=False)